In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import numpy as np



In [ ]:
# Define the paths to the image folder and annotations
images_path = "../Potholes/annotated-images"
annotations_path = "../Potholes/annotated-images"
# Edge Boxes parameters
EDGE_BOXES_PARAMS = {
    "alpha": 0.7,       # Finer granularity in sliding window
    "beta": 0.65,        # Increased sensitivity to edge density
    "minScore": 0.04,   # Minimum score for boxes
    "maxBoxes": 600     # Allow more proposals for potential potholes
}



In [ ]:
# Helper function to parse the XML file for ground truth bounding boxes
def parse_xml(annotation_file):
    tree = ET.parse(annotation_file)
    root = tree.getroot()
    boxes = []
    
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        boxes.append((xmin, ymin, xmax, ymax))
    
    return boxes

In [ ]:
def get_edge_boxes(image, model_path="model.yml", maxBoxes = EDGE_BOXES_PARAMS["maxBoxes"], alpha = EDGE_BOXES_PARAMS["alpha"], beta = EDGE_BOXES_PARAMS["beta"], minScore = EDGE_BOXES_PARAMS["minScore"]):
    # Set up Edge Boxes
    edge_boxes = cv2.ximgproc.createEdgeBoxes()
    edge_boxes.setMaxBoxes(maxBoxes)
    edge_boxes.setAlpha(alpha)
    edge_boxes.setBeta(beta)
    edge_boxes.setMinScore(minScore)
    
    # Load the Structured Edge Detection model
    edge_detector = cv2.ximgproc.createStructuredEdgeDetection(model_path)
    
    # Convert image to RGB and then to 32-bit floating point
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb = image_rgb.astype(np.float32) / 255.0  # Convert to [0,1] range and 32-bit float

    # Detect edges and compute orientation map
    edges = edge_detector.detectEdges(image_rgb)
    orientation_map = edge_detector.computeOrientation(edges)
    
    # Generate Edge Boxes proposals using both edge map and orientation map
    boxes, scores = edge_boxes.getBoundingBoxes(edges, orientation_map)
    edge_boxes_list = [(x, y, w, h) for [x, y, w, h] in boxes]
    return edge_boxes_list


In [ ]:
# Helper function to display an image with ground truth and Edge Boxes proposals
def visualize_image_with_proposals(image_file, gt_boxes, proposals):
    image = cv2.imread(image_file)
    if image is None:
        print(f"Error: Could not load image at {image_file}")
        return
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Draw ground truth boxes in blue
    for (xmin, ymin, xmax, ymax) in gt_boxes:
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
    
    # Draw Edge Boxes proposals in green
    for (x, y, w, h) in proposals:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 1)

    plt.imshow(image)
    plt.axis('off')
    plt.show()

    # recall = calc_recall(proposals, gt_boxes, 0.5)
    # print(f"Recall: {recall}")

In [ ]:
# Main loop to process images, parse XML, and generate Edge Boxes proposals
def displayBoxesWithParams(maxBoxes, alpha, beta, minScore):
    for image_filename in os.listdir(images_path):
        if image_filename.endswith(".jpg") and image_filename == "img-639.jpg":
            image_path = os.path.join(images_path, image_filename)
            xml_filename = image_filename.replace(".jpg", ".xml")
            xml_path = os.path.join(annotations_path, xml_filename)
            
            print("Processing:", image_path)
            
            # Check if XML file exists and parse ground truth boxes
            if os.path.exists(xml_path):
                gt_boxes = parse_xml(xml_path)
            else:
                print(f"Warning: Annotation file {xml_path} does not exist.")
                gt_boxes = []
            
            # Load the image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Could not load image at {image_path}")
                continue
            
            # Generate Edge Boxes proposals
            edge_boxes_proposals = get_edge_boxes(image, "model.yml", maxBoxes, alpha, beta, minScore)
            
            # Visualize both ground truth boxes and Edge Boxes proposals
            visualize_image_with_proposals(image_path, gt_boxes, edge_boxes_proposals)
            
            # Display only a few examples for familiarity
            if len(gt_boxes) > 2:
                break  # Remove this line if you want to display more images
displayBoxesWithParams(600,0.7,0.65,0.04)

In [ ]:
# Base parameter values
base_alpha = 0.7
base_beta = 0.65
base_minScore = 0.04
maxBoxes = 600

# Parameter ranges around the base values
#alpha_values = [base_alpha - 0.1, base_alpha, base_alpha + 0.1]
beta_values = [0.7]
minScore_values = [0.02, 0.04, 0.05, 0.07]

# Loop through each combination of parameters
#displayBoxesWithParams(maxBoxes, base_alpha, base_beta, base_minScore)
# for beta in beta_values:
#     for minScore in minScore_values:
#         print(f"Trying parameters: beta={beta}, minScore={minScore} -----------------------------------------------------------------------------------------")
#         displayBoxesWithParams(maxBoxes, base_alpha, beta, minScore)

In [ ]:
# # Base parameter values
# base_alpha = 0.8
# base_beta = 0.6
# base_minScore = 0.05
# maxBoxes = 200

# # Parameter ranges around the base values
# #alpha_values = [base_alpha - 0.1, base_alpha, base_alpha + 0.1]
# beta_values = [base_beta - 0.1, base_beta, base_beta + 0.1]
# minScore_values = [base_minScore - 0.02, base_minScore, base_minScore + 0.02]

# # Loop through each combination of parameters

# for beta in beta_values:
#     for minScore in minScore_values:
#         print(f"Trying parameters: beta={beta}, minScore={minScore}")
#         displayBoxesWithParams(maxBoxes, base_alpha, beta, minScore)

## TO DO: IMPLEMENT METRICS

IOU, Recall, and ABO (courtesy of Alba ty <3>)

In [ ]:
# Constants
IOU_THRESHOLD = 0.5  # For recall calculation
TARGET_PROPOSALS = range(10, 200, 10)  # Range of number of proposals to test (adjust as needed)

# Function to calculate the IoU between to proposals(two boxes)
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxA_Area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxB_Area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxA_Area + boxB_Area - interArea)
    return iou

# Function to evaluate proposals on a single image using recall
def calc_recall(proposals, ground_truth_boxes, iou_threshold):
    recalled_boxes = 0
    for gt_box in ground_truth_boxes:
        max_iou = 0  # Variable to store the maximum IoU for this ground truth box
        for (x, y, w, h) in proposals:
            iou = calculate_iou(gt_box, (x, y, x + w, y + h))
            if iou > max_iou:
                max_iou = iou  # Update if a higher IoU is found
        # Only count if the maximum IoU exceeds the threshold
        if max_iou >= iou_threshold:
            recalled_boxes += 1
    recall = recalled_boxes / len(ground_truth_boxes)
    return recall

# Function to evaluate proposals on a single image using ABO
def calc_abo(proposals, ground_truth_boxes):
    sum_max_ious = 0
    for gt_box in ground_truth_boxes:
        max_iou = 0
        for (x, y, w, h) in proposals:
            iou = calculate_iou(gt_box, (x, y, x + w, y + h))
            max_iou= max(max_iou, iou)

        sum_max_ious += max_iou
    
    abo = sum_max_ious / len(ground_truth_boxes) 
    return abo
    

### Loop for evaluating the Recall and choosing the number of proposals



In [ ]:
# # Main loop to process images, parse XML, and generate Edge Boxes proposals
# iou_thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]

# # Define max number of proposals to use (e.g., 100)
# max_proposals = 100

# def displayBoxesWithParamsEval(maxBoxes, alpha, beta, minScore, max_proposals):
#     for image_filename in os.listdir(images_path):
#         if image_filename.endswith(".jpg"):
#             image_path = os.path.join(images_path, image_filename)
#             xml_filename = image_filename.replace(".jpg", ".xml")
#             xml_path = os.path.join(annotations_path, xml_filename)
            
#             print("Processing:", image_path)
            
#             # Check if XML file exists and parse ground truth boxes
#             if os.path.exists(xml_path):
#                 gt_boxes = parse_xml(xml_path)
#             else:
#                 print(f"Warning: Annotation file {xml_path} does not exist.")
#                 gt_boxes = []
            
#             # Load the image
#             image = cv2.imread(image_path)
#             if image is None:
#                 print(f"Error: Could not load image at {image_path}")
#                 continue
            
#             # Generate Edge Boxes proposals
#             edge_boxes_proposals = get_edge_boxes(image, "model.yml", maxBoxes, alpha, beta, minScore)
#             limited_proposals = edge_boxes_proposals[:max_proposals]
#             # Visualize both ground truth boxes and Edge Boxes proposals
#             #visualize_image_with_proposals(image_path, gt_boxes, edge_boxes_proposals)

#             recalls = []
#             for iou_threshold in iou_thresholds:
#                 recall = calc_recall(limited_proposals, gt_boxes, iou_threshold)
#                 recalls.append(recall)
            
#                 # Calculate Average Recall (AR) over the thresholds
#             average_recall = sum(recalls) / len(iou_thresholds)
            
#             # Calculate Average Best Overlap (ABO) using all proposals
#             abo = calc_abo(edge_boxes_proposals, gt_boxes)
            
#             results = {
#                 "average_recall": average_recall,
#                 "recalls_at_thresholds": dict(zip(iou_thresholds, recalls)),
#                 "average_best_overlap": abo
#             }
            
#             print("Average Recall:", results["average_recall"])
#             print("Recall at IoU thresholds:", results["recalls_at_thresholds"])
#             print("Average Best Overlap:", results["average_best_overlap"])
#             # Display only a few examples for familiarity
#             if len(gt_boxes) > 2:
#                 break  # Remove this line if you want to display more images

# base_alpha = 0.85
# base_beta = 0.6
# base_minScore = 0.05
# maxBoxes = 75

# displayBoxesWithParamsEval(maxBoxes, base_alpha, base_beta, base_minScore, 100)

In [ ]:
import matplotlib.pyplot as plt

#iou_thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]
iou_thresholds = [0.5]
# Define max number of proposals to use (e.g., 100)
#max_proposals = 100
numImagesPer = 1

def displayBoxesWithParamsEval(maxBoxes, alpha, beta, minScore, max_proposals):
    all_recalls = {iou: [] for iou in iou_thresholds}
    all_abos = []
    imageCounter = 1
    for image_filename in os.listdir(images_path):
        if image_filename.endswith(".jpg"):
            image_path = os.path.join(images_path, image_filename)
            xml_filename = image_filename.replace(".jpg", ".xml")
            xml_path = os.path.join(annotations_path, xml_filename)
            
            print(f"Processing: {image_path}, image {imageCounter}/{numImagesPer}          ", end='\r')
            
            # Check if XML file exists and parse ground truth boxes
            if os.path.exists(xml_path):
                gt_boxes = parse_xml(xml_path)
            else:
                print(f"Warning: Annotation file {xml_path} does not exist.")
                gt_boxes = []
            
            # Load the image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Could not load image at {image_path}")
                continue
            
            # Generate Edge Boxes proposals
            edge_boxes_proposals = get_edge_boxes(image, "model.yml", maxBoxes, alpha, beta, minScore)
            limited_proposals = edge_boxes_proposals[:max_proposals]
            
            # Calculate recall for each IoU threshold
            for iou_threshold in iou_thresholds:
                recall = calc_recall(limited_proposals, gt_boxes, iou_threshold)
                all_recalls[iou_threshold].append(recall)
            
            # Calculate Average Best Overlap (ABO)
            abo = calc_abo(edge_boxes_proposals, gt_boxes)
            all_abos.append(abo)

            # Visualize both ground truth boxes and Edge Boxes proposals
            #visualize_image_with_proposals(image_path, gt_boxes, edge_boxes_proposals)
            imageCounter += 1
            
            if imageCounter > numImagesPer:
                print(f"Processing: {image_path}, image {imageCounter}/{numImagesPer}")
                break

    # Calculate average recall for each IoU threshold
    average_recalls = {iou: sum(recalls) / len(recalls) for iou, recalls in all_recalls.items()}
    
    # Calculate overall Average Recall (AR) across all thresholds
    overall_average_recall = sum(average_recalls.values()) / len(iou_thresholds)
    
    # Calculate overall Average Best Overlap (ABO) across all images
    overall_abo = sum(all_abos) / len(all_abos)
    print(f"For Params: alpha = {alpha}, beta = {beta}, minScore = {minScore}, maxBoxes = {maxBoxes}")
    print("Average Recall across all images:", overall_average_recall)
    print("Recall at IoU thresholds across all images:", average_recalls)
    print("Average Best Overlap across all images:", overall_abo)
    
    return overall_abo, average_recalls  # Return ABO for later comparison


# List to store the ABOS along with their respective parameters
all_results = []
minScores = []  # Store minScores for plotting
alphas = []
betas = []
maxBoxesfinal = []
abos = []  # Store ABOS for plotting
recalls = []
alpha_values = [0.7]
beta_values = [0.65]
minScore_values = [0.004]
maxBox_values = [600]
#[0.005, 0.01, 0.015, 0.02, 0.03, 0.04, 0.05, 0.1, 0.2]
# Loop through each combination of parameters
for alpha in alpha_values:
    for beta in beta_values:
        for minScore in minScore_values:
            for maxBoxes in maxBox_values:
                #print(f"Trying parameters: alpha={alpha}, beta={beta}, minScore={minScore} -----------------------------------------------------------------------------------------")
                abo, avg_recalls = displayBoxesWithParamsEval(maxBoxes, alpha, beta, minScore, 100)
                
                # Store the ABO with its parameters
                all_results.append((alpha, beta, minScore, abo, avg_recalls))
                minScores.append(minScore)  # Store minScore for plotting
                alphas.append(alpha)
                betas.append(beta)
                abos.append(abo)  # Store ABO for plotting
                recalls.append(avg_recalls[0.5])
                maxBoxesfinal.append(maxBoxes)

# Find the best ABO and corresponding parameters
best_result = max(all_results, key=lambda x: x[3])  # Get the tuple with the highest ABO

# Print the best result
print("\nBest ABO:")
print(f"alpha = {best_result[0]}, beta = {best_result[1]}, minScore = {best_result[2]}, ABO = {best_result[3]}, Avg Recall 0.5 = {best_result[4][0.5]}")

# # Plot minScore vs ABO
# plt.figure(figsize=(8, 6))
# plt.plot(alphas, recalls, marker='o', linestyle='-', color='b', label='ABO')
# plt.xlabel('alpha')
# plt.ylabel('Average Recall )')
# plt.title('alpha vs Recall')
# plt.grid(True)
# plt.legend()
# plt.show()


In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET

EDGE_BOXES_PARAMS = {
    "alpha": 0.8,       # Finer granularity in sliding window
    "beta": 0.7,        # Increased sensitivity to edge density
    "minScore": 0.04,   # Minimum score for boxes
    "maxBoxes": 600     # Allow more proposals for potential potholes
}

def create_xml_from_proposals(image_filename, proposals, output_folder):
    # Create a new XML structure for the proposals
    root = ET.Element("annotation")
    folder = ET.SubElement(root, "folder")
    folder.text = "Proposals"

    filename = ET.SubElement(root, "filename")
    filename.text = image_filename

    size = ET.SubElement(root, "size")
    width = ET.SubElement(size, "width")
    height = ET.SubElement(size, "height")
    depth = ET.SubElement(size, "depth")

    # Assuming you know the image size (for simplicity, using fixed values)
    image = cv2.imread(os.path.join(images_path, image_filename))
    height.text = str(image.shape[0])
    width.text = str(image.shape[1])
    depth.text = str(image.shape[2])

    # Add the proposals as objects
    for idx, proposal in enumerate(proposals):
        object_elem = ET.SubElement(root, "object")
        name = ET.SubElement(object_elem, "name")
        name.text = "proposal"  # You can use a more descriptive name if needed

        bndbox = ET.SubElement(object_elem, "bndbox")
        xmin = ET.SubElement(bndbox, "xmin")
        ymin = ET.SubElement(bndbox, "ymin")
        xmax = ET.SubElement(bndbox, "xmax")
        ymax = ET.SubElement(bndbox, "ymax")

        xmin.text = str(int(proposal[0]))  # x1 coordinate
        ymin.text = str(int(proposal[1]))  # y1 coordinate
        xmax.text = str(int(proposal[2]))  # x2 coordinate
        ymax.text = str(int(proposal[3]))  # y2 coordinate

    # Save XML to output folder
    tree = ET.ElementTree(root)
    output_xml_path = os.path.join(output_folder, image_filename.replace(".jpg", "_proposals.xml"))
    tree.write(output_xml_path)

def generate_proposals_and_save_xml(maxBoxes = 600, alpha = 0.7, beta = 0.65, minScore = 0.04, output_folder="edge_box_proposals"):
    # Get the parent directory of the current file
    parent_directory = os.getcwd()  # Get the absolute path of the current file's directory
    output_folder_path = os.path.join(parent_directory, output_folder)  # Join it with the output folder name
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    counter = 1
    total = int(len(os.listdir(images_path))/2)
    for image_filename in os.listdir(images_path):
        if image_filename.endswith(".jpg"):
            image_path = os.path.join(images_path, image_filename)
            
            print(f"Processing: {image_path}, {counter}/{total}              ", end='\r')
            
            # Load the image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Could not load image at {image_path}")
                continue

            # Generate Edge Boxes proposals
            edge_boxes_proposals = get_edge_boxes(image, "model.yml", maxBoxes, alpha, beta, minScore)

            # Create XML file with proposals
            create_xml_from_proposals(image_filename, edge_boxes_proposals, output_folder)

            #print(f"Generated XML for {image_filename} in {output_folder}")
            counter += 1
            # if counter > 10:
            #     break
generate_proposals_and_save_xml()

In [ ]:
current_dir = os.path.join(os.getcwd(), "edge_box_proposals" )
xml_path = os.path.join(current_dir, "img-639_proposals.xml")
image_path = os.path.join(images_path, "img-639.jpg")

print("Processing:", image_path)

# Check if XML file exists and parse ground truth boxes
if os.path.exists(xml_path):
    gt_boxes = parse_xml(xml_path)
else:
    print(f"Warning: Annotation file {xml_path} does not exist.")
    gt_boxes = []

# Load the image
image = cv2.imread(image_path)
if image is None:
    print(f"Error: Could not load image at {image_path}")

# Generate Edge Boxes proposals
#edge_boxes_proposals = get_edge_boxes(image, "model.yml", maxBoxes, alpha, beta, minScore)
edge_boxes_proposals = []
# Visualize both ground truth boxes and Edge Boxes proposals
visualize_image_with_proposals(image_path, [], gt_boxes)